In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

https://www.youtube.com/watch?v=1-NYPQw5THU&feature=youtu.be

In [2]:
import pandas as pd
import numpy as np
import datetime
from pandas_summary import DataFrameSummary

In [3]:
df = pd.read_feather('train_normalized_data.fth')
df_test = pd.read_feather('test_normalized_data.fth')

In [4]:
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen', 'Promo2Weeks', 
            'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear', 'State', 
            'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_bool_fw', 'StateHoliday_bool_bw', 'SchoolHoliday_fw', 'SchoolHoliday_bw']

# cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'State']


In [5]:
contin_vars = ['CompetitionDistance', 
   'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
   'AfterStateHoliday_bool', 'BeforeStateHoliday_bool', 'Promo', 'SchoolHoliday']
# contin_vars = []

In [6]:
from lightgbm import LGBMRegressor

In [7]:
y_out_columns = ['Sales']

In [8]:
df_train = df[df.Date < datetime.datetime(2015, 7, 1)]  
df_val = df[df.Date >= datetime.datetime(2015, 7, 1)]
print(f'Cantidad en val: {len(df_val)}, porcentaje: {len(df_train)/(len(df_train) + len(df_val))}')

Cantidad en val: 30188, porcentaje: 0.9642465458145908


In [9]:
X_train = df_train[cat_vars + contin_vars]
X_val = df_val[cat_vars + contin_vars]
X_test = df_test[cat_vars + contin_vars]

In [10]:
X_train.shape, X_val.shape

((814150, 38), (30188, 38))

In [11]:
log_output = True
    
if log_output:
    # Escala logaritmica
    max_log_y = np.max(np.log(df[y_out_columns])).values
    y_train = np.log(df_train[y_out_columns].values)/max_log_y
    y_val = np.log(df_val[y_out_columns].values)/max_log_y
else:
    # Normalización
    y_mean = df_train[y_out_columns].mean().values
    y_std = df_train[y_out_columns].std().values
    y_train = (df_train[y_out_columns].values - y_mean)/y_std
    y_val = (df_val[y_out_columns].values - y_mean)/y_std

In [12]:
min_child_samples=5
n_estimators=4000
learning_rate=0.05
model = LGBMRegressor(min_child_samples=min_child_samples, n_estimators=n_estimators, learning_rate=learning_rate )

In [13]:
fit_params={"early_stopping_rounds":100, 
            "eval_metric" : 'l2', 
            "eval_set" : [(X_val, y_val.reshape(-1))],
            'eval_names': ['valid'],
            'verbose': 100,
            'feature_name': 'auto', # that's actually the default
            'categorical_feature': cat_vars
           }

In [14]:
model.fit(X_train, y_train.reshape(-1), **fit_params)

C:\Users\Patricio\anaconda3\envs\curso\lib\site-packages\lightgbm\basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\Patricio\anaconda3\envs\curso\lib\site-packages\lightgbm\basic.py:1705: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Assortment', 'CompetitionMonthsOpen', 'CompetitionOpenSinceYear', 'Day', 'DayOfWeek', 'Events', 'Month', 'Promo2SinceYear', 'Promo2Weeks', 'PromoInterval', 'Promo_bw', 'Promo_fw', 'SchoolHoliday_bw', 'SchoolHoliday_fw', 'State', 'StateHoliday', 'StateHoliday_bool_bw', 'StateHoliday_bool_fw', 'Store', 'StoreType', 'Week', 'Year']
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\Users\Patricio\anaconda3\envs\curso\lib\site-packages\lightgbm\basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\Patricio\anaconda3\envs\cu

Training until validation scores don't improve for 100 rounds
[100]	valid's l2: 0.000223293
[200]	valid's l2: 0.000173169
[300]	valid's l2: 0.000152065
[400]	valid's l2: 0.000145942
[500]	valid's l2: 0.000139037
[600]	valid's l2: 0.000137416
Early stopping, best iteration is:
[586]	valid's l2: 0.000136971


LGBMRegressor(learning_rate=0.05, min_child_samples=5, n_estimators=4000)

# Métrica

In [15]:
display(DataFrameSummary(X_val).summary())

,Store,DayOfWeek,Year,Month,Day,StateHoliday,CompetitionMonthsOpen,Promo2Weeks,StoreType,Assortment,...,Min_Humidity,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,CloudCover,trend,trend_DE,AfterStateHoliday_bool,BeforeStateHoliday_bool,Promo,SchoolHoliday
count,30188.0,30188.0,30188.0,30188.0,30188.0,30188.0,30188.0,30188.0,30188.0,30188.0,...,30188.0,30188.0,30188.0,30188.0,30188.0,30188.0,30188.0,30188.0,30188.0,30188.0
mean,556.949549,2.570227,2.0,6.0,15.06761,0.0,22.358222,24.893037,1.217139,0.930005,...,-0.854232,0.246864,0.008632,-0.332594,1.195729,1.55777,0.266208,1.072424,0.201434,0.63466
std,321.784839,1.652367,0.0,0.0,9.092351,0.0,4.989704,1.261481,1.367624,0.992921,...,0.673537,1.057187,0.960515,1.0112,0.790217,0.592071,0.327174,0.0,1.028458,1.289232
min,0.0,0.0,2.0,6.0,0.0,0.0,0.0,4.0,0.0,0.0,...,-2.034175,-1.305635,-1.50424,-2.314268,-1.430281,0.807334,-0.317714,1.072424,-0.7854,-0.466372
25%,278.0,1.0,2.0,6.0,7.0,0.0,24.0,25.0,0.0,0.0,...,-1.320848,-0.522673,-0.656379,-0.94896,0.686147,0.807334,0.002983,1.072424,-0.7854,-0.466372
50%,557.0,3.0,2.0,6.0,15.0,0.0,24.0,25.0,0.0,0.0,...,-0.964185,0.148437,-0.147662,-0.038755,1.215254,1.743049,0.29161,1.072424,-0.7854,-0.466372
75%,835.0,4.0,2.0,6.0,23.0,0.0,24.0,25.0,3.0,2.0,...,-0.454666,0.707695,0.7002,0.416347,1.832546,2.054953,0.516098,1.072424,1.273237,2.144211
max,1114.0,6.0,2.0,6.0,30.0,0.0,24.0,25.0,3.0,2.0,...,1.328652,5.069909,3.413358,1.326552,2.361653,2.158922,0.965073,1.072424,1.273237,2.144211
counts,30188,30188,30188,30188,30188,30188,30188,30188,30188,30188,...,30188,30188,30188,30188,30188,30188,30188,30188,30188,30188
uniques,1115,7,1,1,31,1,25,19,4,3,...,60,26,18,9,32,4,41,1,2,2


$$
\textrm{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} \left(\frac{\hat{y}_i - y_i}{y_i}\right)^2}
$$

In [16]:
model.score(X_val, y_val)

0.9040491238523836

In [17]:
if log_output:
    y_pred_train = np.exp(model.predict(X_train, verbose=1)*max_log_y)
    y_pred = np.exp(model.predict(X_val, verbose=1)*max_log_y)
    y_pred_test = np.exp(model.predict(X_test, verbose=1)*max_log_y)
else:
    y_pred_train = model.predict(X_train, verbose=1)*y_std + y_mean
    y_pred = model.predict(X_val, verbose=1)*y_std + y_mean
    y_pred_test = model.predict(X_test, verbose=1)*y_std + y_mean

In [18]:
# Train
np.sqrt((((df_train['Sales'].values - y_pred_train)/df_train['Sales'].values)**2).sum()/len(y_pred_train))

0.1507151254495438

In [19]:
# Validación
np.sqrt((((df_val['Sales'].values - y_pred)/df_val['Sales'].values)**2).sum()/len(y_pred))

0.12971132708335936

# Baseline

In [20]:
import pandas as pd
sample_csv = pd.read_csv('dataset/rossmann/sample_submission.csv')

In [21]:
stores_mean = {}
for store, g_df in df.groupby('Store'):
    stores_mean[store] = g_df[g_df['Sales'] > 0]['Sales'].mean()

In [22]:
df_test['Sales'] = df_test['Store'].apply(stores_mean.get)
df_test.loc[df_test['Open'] == 0, 'Sales'] = 0

In [23]:
df_test[['Store', 'Sales']].head(10)

,Store,Sales
0,0,4759.096031
1,2,6942.568678
2,6,8817.050891
3,7,5539.358418
4,8,6562.337612
5,9,5568.420918
6,10,8030.977041
7,11,7589.598214
8,12,5034.747182
9,13,5508.567394


In [24]:
df_test[df_test['Open'] == 0][['Store', 'Sales']].head()

,Store,Sales
543,702,0.0
676,878,0.0
840,1096,0.0
1399,702,0.0
1532,878,0.0


In [25]:
sample_csv['Sales'] = df_test['Sales']

In [26]:
sample_csv.to_csv(f'submision_baseline.csv', index=False)

In [27]:
sample_csv.head()

,Id,Sales
0,1,4759.096031
1,2,6942.568678
2,3,8817.050891
3,4,5539.358418
4,5,6562.337612


# Sumbit a la competición

In [28]:
sample_csv = pd.read_csv('dataset/rossmann/sample_submission.csv')
sample_csv['Sales'] = y_pred_test
sample_csv.head()

sample_csv.to_csv('submit_lgbm_kaggle_02_sincerrados.csv', index=False)